# 03 - Train Model

This notebook provides an interactive training interface:
- Adjust hyperparameters interactively
- Visualize training curves in real-time
- Try different model configurations
- Monitor training progress


In [ ]:
# Import libraries
import os
import sys
import json
import torch
import torch.nn as nn
from pathlib import Path
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score, precision_recall_fscore_support
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time

# Add src to path
sys.path.insert(0, str(Path().absolute().parent / 'src'))

from data_processor import DFGDataset, load_config, load_dfg_mapping
from model import DFGClassifier

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

# Enable interactive plots
%matplotlib inline

print("✓ Libraries imported")


In [ ]:
# Configuration
DATA_PATH = '../dfg-classifier/data/processed'  # Change to '../dfg-classifier/data/small' for small dataset
CONFIG_PATH = '../config.yaml'
DFG_MAPPING_PATH = '../data/dfg_mapping.json'
OUTPUT_DIR = '../dfg-classifier/models/experiments'

# Load configuration
config = load_config(CONFIG_PATH)
dfg_mapping = load_dfg_mapping(DFG_MAPPING_PATH)

# Create output directory
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("✓ Configuration loaded")


In [ ]:
# Load datasets
def load_processed_dataset(data_path: str, split: str = 'train'):
    """Load processed dataset"""
    file_path = os.path.join(data_path, f'{split}.json')
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Dataset file not found: {file_path}")
    
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # Convert back to tensors
    dataset = []
    for item in data:
        dataset.append({
            'input_ids': torch.tensor(item['input_ids']),
            'attention_mask': torch.tensor(item['attention_mask']),
            'labels': torch.tensor(item['labels']),
            'filename': item['filename'],
            'title': item['title'],
            'abstract': item['abstract'],
            'label': item['label']
        })
    
    return dataset

print("📊 Loading datasets...")
train_dataset = load_processed_dataset(DATA_PATH, 'train')
val_dataset = load_processed_dataset(DATA_PATH, 'val')
test_dataset = load_processed_dataset(DATA_PATH, 'test')

print(f"✓ Train: {len(train_dataset)} samples")
print(f"✓ Val: {len(val_dataset)} samples")
print(f"✓ Test: {len(test_dataset)} samples")


## Hyperparameters

Adjust these parameters to experiment with different training configurations:


In [ ]:
# Hyperparameters - Adjust these!
BATCH_SIZE = 4  # Try 2, 4, 8, 16
NUM_EPOCHS = 3  # Try 3, 5, 10
LEARNING_RATE = 0.00002  # Try 1e-5, 2e-5, 5e-5
DROPOUT_RATE = 0.3  # Try 0.1, 0.3, 0.5
FREEZE_BERT = False  # Set True to freeze BERT weights
MODEL_NAME = 'allenai/scibert_scivocab_uncased'  # Pre-trained model

# Training settings
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
PRINT_EVERY = 10  # Print progress every N batches

print("⚙️  Training Configuration:")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Epochs: {NUM_EPOCHS}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Dropout: {DROPOUT_RATE}")
print(f"  Freeze BERT: {FREEZE_BERT}")
print(f"  Device: {DEVICE}")


In [ ]:
# Create data loaders
def create_data_loader(dataset, batch_size, shuffle=False):
    """Create PyTorch DataLoader"""
    torch_dataset = DFGDataset(dataset)
    return DataLoader(
        torch_dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=0,  # 0 for notebook compatibility
        pin_memory=False
    )

train_loader = create_data_loader(train_dataset, BATCH_SIZE, shuffle=True)
val_loader = create_data_loader(val_dataset, BATCH_SIZE, shuffle=False)
test_loader = create_data_loader(test_dataset, BATCH_SIZE, shuffle=False)

print(f"✓ Data loaders created")
print(f"  Train batches: {len(train_loader)}")
print(f"  Val batches: {len(val_loader)}")
print(f"  Test batches: {len(test_loader)}")


In [ ]:
# Initialize model
print("🧠 Initializing model...")

num_classes = len(dfg_mapping['level_2']['classes'])

model = DFGClassifier(
    model_name=MODEL_NAME,
    num_classes=num_classes,
    dropout_rate=DROPOUT_RATE,
    freeze_bert=FREEZE_BERT,
    hierarchical=False,
    dfg_mapping=dfg_mapping
)

model.to(DEVICE)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"✓ Model initialized")
print(f"  Total parameters: {total_params:,}")
print(f"  Trainable parameters: {trainable_params:,}")
print(f"  Num classes: {num_classes}")

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
print(f"✓ Optimizer: Adam (lr={LEARNING_RATE})")


## Training Loop

Run the training loop and monitor progress in real-time.


In [ ]:
# Initialize training history
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': [],
    'val_f1': []
}

best_val_acc = 0.0
best_model_state = None

print("🏋️  Starting training...")
print("=" * 80)

for epoch in range(NUM_EPOCHS):
    epoch_start_time = time.time()
    
    # Training phase
    model.train()
    train_loss = 0
    train_correct = 0
    train_total = 0
    
    train_progress = tqdm(train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Train]")
    for batch_idx, batch in enumerate(train_progress):
        input_ids = batch['input_ids'].to(DEVICE)
        attention_mask = batch['attention_mask'].to(DEVICE)
        labels = batch['labels'].to(DEVICE)
        
        # Forward pass
        outputs = model(input_ids, attention_mask, labels)
        loss = outputs['loss']
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Update metrics
        train_loss += loss.item()
        predictions = outputs['predictions']
        train_correct += (predictions == labels).sum().item()
        train_total += labels.size(0)
        
        # Update progress bar
        train_progress.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{train_correct/train_total:.4f}'
        })
    
    avg_train_loss = train_loss / len(train_loader)
    train_acc = train_correct / train_total
    
    # Validation phase
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        val_progress = tqdm(val_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Val]")
        for batch in val_progress:
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)
            
            outputs = model(input_ids, attention_mask, labels)
            loss = outputs['loss']
            predictions = outputs['predictions']
            
            val_loss += loss.item()
            val_correct += (predictions == labels).sum().item()
            val_total += labels.size(0)
            
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            val_progress.set_postfix({
                'loss': f'{loss.item():.4f}',
                'acc': f'{val_correct/val_total:.4f}'
            })
    
    avg_val_loss = val_loss / len(val_loader)
    val_acc = val_correct / val_total
    val_f1 = f1_score(all_labels, all_preds, average='weighted', zero_division=0)
    
    # Update history
    history['train_loss'].append(avg_train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(avg_val_loss)
    history['val_acc'].append(val_acc)
    history['val_f1'].append(val_f1)
    
    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = model.state_dict().copy()
    
    epoch_time = time.time() - epoch_start_time
    
    # Print epoch summary
    print(f"\n📊 Epoch {epoch+1}/{NUM_EPOCHS} Summary:")
    print(f"  Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Val Loss: {avg_val_loss:.4f} | Val Acc: {val_acc:.4f} | Val F1: {val_f1:.4f}")
    print(f"  Time: {epoch_time:.2f}s")
    print("-" * 80)

print("\n✅ Training completed!")
print(f"🎯 Best Validation Accuracy: {best_val_acc:.4f}")

# Load best model
if best_model_state:
    model.load_state_dict(best_model_state)
    print("✓ Loaded best model state")


In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

epochs = range(1, len(history['train_loss']) + 1)

# Loss curve
axes[0].plot(epochs, history['train_loss'], 'o-', label='Train Loss', color='steelblue', linewidth=2)
axes[0].plot(epochs, history['val_loss'], 's-', label='Val Loss', color='coral', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Accuracy curve
axes[1].plot(epochs, history['train_acc'], 'o-', label='Train Acc', color='steelblue', linewidth=2)
axes[1].plot(epochs, history['val_acc'], 's-', label='Val Acc', color='coral', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(alpha=0.3)
axes[1].set_ylim([0, 1])

# F1 score curve
axes[2].plot(epochs, history['val_f1'], 's-', label='Val F1', color='green', linewidth=2)
axes[2].set_xlabel('Epoch', fontsize=12)
axes[2].set_ylabel('F1 Score', fontsize=12)
axes[2].set_title('Validation F1 Score', fontsize=14, fontweight='bold')
axes[2].legend()
axes[2].grid(alpha=0.3)
axes[2].set_ylim([0, 1])

plt.tight_layout()
plt.show()

# Print final metrics
print("\n📈 Final Training Metrics:")
print(f"  Train Loss: {history['train_loss'][-1]:.4f}")
print(f"  Train Accuracy: {history['train_acc'][-1]:.4f}")
print(f"  Val Loss: {history['val_loss'][-1]:.4f}")
print(f"  Val Accuracy: {history['val_acc'][-1]:.4f}")
print(f"  Val F1: {history['val_f1'][-1]:.4f}")
print(f"  Best Val Accuracy: {best_val_acc:.4f}")


## Save Model

Save the trained model for later use.


In [ ]:
# Save model and training history
model_path = os.path.join(OUTPUT_DIR, 'trained_model.pt')
torch.save(model.state_dict(), model_path)

history_path = os.path.join(OUTPUT_DIR, 'training_history.json')
with open(history_path, 'w') as f:
    json.dump(history, f, indent=2)

print(f"✓ Model saved to: {model_path}")
print(f"✓ Training history saved to: {history_path}")

# Save training config
training_config = {
    'batch_size': BATCH_SIZE,
    'num_epochs': NUM_EPOCHS,
    'learning_rate': LEARNING_RATE,
    'dropout_rate': DROPOUT_RATE,
    'freeze_bert': FREEZE_BERT,
    'model_name': MODEL_NAME,
    'best_val_acc': best_val_acc,
    'total_params': total_params,
    'trainable_params': trainable_params
}

config_path = os.path.join(OUTPUT_DIR, 'training_config.json')
with open(config_path, 'w') as f:
    json.dump(training_config, f, indent=2)

print(f"✓ Training config saved to: {config_path}")
